In [1]:
import cupy

print(cupy.show_config())


OS                           : Linux-6.14.0-28-generic-x86_64-with-glibc2.39
Python Version               : 3.12.3
CuPy Version                 : 13.6.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 2.3.2
SciPy Version                : None
Cython Build Version         : 3.0.12
Cython Runtime Version       : None
CUDA Root                    : /usr/local/cuda-12.8
nvcc PATH                    : /usr/local/cuda-12.8/bin/nvcc
CUDA Build Version           : 12090
CUDA Driver Version          : 12080
CUDA Runtime Version         : 12090 (linked to CuPy) / 12080 (locally installed)
CUDA Extra Include Dirs      : ['/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/nvidia/cuda_runtime/include']
cuBLAS Version               : (available)
cuFFT Version                : 11303
cuRAND Version               : 10309
cuSOLVER Version             : (11, 7, 3)
cuSPARSE Version             : (available)
NVRTC Version                : (12, 8)
Thrust Version    

In [2]:
# Cell 1: 
import spacy 
from pathlib import Path 
import os 
# 查看 spaCy 版本和可用组件 
spacy.info()

{'spacy_version': '3.8.7',
 'location': '/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/spacy',
 'platform': 'Linux-6.14.0-28-generic-x86_64-with-glibc2.39',
 'python_version': '3.12.3',
 'pipelines': {}}

In [3]:
# Cell 2:
# 创建 corpus 和 models 文件夹
Path("./corpus").mkdir(parents=True, exist_ok=True)
Path("./models").mkdir(parents=True, exist_ok=True)


In [4]:
# Cell 3:
!python -m spacy convert ud_latvian/lv_lvtb-ud-train.conllu ./corpus
!python -m spacy convert ud_latvian/lv_lvtb-ud-dev.conllu ./corpus
!python -m spacy convert ud_latvian/lv_lvtb-ud-test.conllu ./corpus


ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (15055 documents):
corpus/lv_lvtb-ud-train.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (2080 documents):
corpus/lv_lvtb-ud-dev.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (2396 documents):
corpus/lv_lvtb-ud-test.spacy


In [5]:
#Cell 4: 
!python -m spacy init config config.cfg --lang lv --pipeline tok2vec,tagger,morphologizer,parser --optimize efficiency


ℹ Generated config template specific for your use case
- Language: lv
- Pipeline: tagger, morphologizer, parser
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
# Cell 5: 
cfg_text = Path("config.cfg").read_text(encoding="utf-8")
cfg_text = cfg_text.replace("train = null", "train = ./corpus/lv_lvtb-ud-train.spacy")
cfg_text = cfg_text.replace("dev = null", "dev = ./corpus/lv_lvtb-ud-dev.spacy")
Path("config.cfg").write_text(cfg_text, encoding="utf-8")
print("✅ 已经修改 config.cfg 的数据路径")


✅ 已经修改 config.cfg 的数据路径


In [7]:
# Cell 6: 使用 GPU 训练 
!python -m spacy train config.cfg --output ./models --paths.train ./corpus/lv_lvtb-ud-train.spacy --paths.dev ./corpus/lv_lvtb-ud-dev.spacy --gpu-id 0

ℹ Saving to output directory: models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger', 'morphologizer', 'parser']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  LOSS MORPH...  LOSS PARSER  TAG_ACC  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_F  SCORE 
---  ------  ------------  -----------  -------------  -----------  -------  -------  ---------  -------  -------  -------  ------
  0       0          0.00        87.49          87.46       278.06    14.64    25.48      15.06     7.89     6.41     0.08    0.14
  0     200       2697.94     13466.27       12736.27     24679.18    45.74    74.21      52.00    49.74    30.81    77.26    0.50
  0     400       5505.20     12785.04       11611.23     23291.23    60.86    83.65      68.25    59.64    43.57    87.89    0.63
  0     600       7751.

In [8]:
# Cell 7: 
!python -m spacy evaluate ./models/model-best ./corpus/lv_lvtb-ud-test.spacy


ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK      99.53
TAG      87.06
POS      94.98
MORPH    90.93
UAS      81.52
LAS      75.78
SENT P   99.75
SENT R   99.87
SENT F   99.81
SPEED    14171


============================== MORPH (per feat) ==============================

                P        R        F
ExtPos      92.86    84.17    88.30
Case        92.19    91.75    91.97
Gender      93.61    93.10    93.35
Number      94.13    93.10    93.61
Person      95.34    94.09    94.71
PronType    98.31    97.94    98.13
Evident     94.92    93.43    94.17
Mood        94.30    92.42    93.35
Polarity    94.84    93.28    94.05
Tense       91.51    89.35    90.42
VerbForm    93.30    91.75    92.52
Voice       94.45    91.93    93.17
Definite    90.98    89.17    90.07
Degree      93.19    91.65    92.42
Poss       100.00   100.00   100.00
NumType     98.84    76.91    86.51
Reflex      92.